<div class="frontmatter text-center">
<h1> Introduction to Data Science and Programming</h1>
<h2>Lecture 26: The data science process</h2>
<h3>IT University of Copenhagen, Fall 2020</h3>
<h3>Instructor: Michael Szell</h3>
</div>

# Source
This notebook was adapted from:
* Joel Grus: Data Science from Scratch

In [1]:
import csv
import pprint as pp # pretty print

# Parsing csv files

### Reading a clean data set

In [2]:
!head files/stockprices.csv

6/20/2014,AAPL,90.91
6/20/2014,MSFT,41.68
6/20/2014,FB,64.5
6/19/2014,AAPL,91.86
6/19/2014,MSFT,41.51
6/19/2014,FB,64.34

Let's read this with the simplest way (using csv.reader):

In [3]:
data = []
with open('files/stockprices.csv', 'r') as f: 
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        date = row[0]
        symbol = row[1]
        closing_price = float(row[2]) 
        data.append([date,symbol,closing_price])
pp.pprint(data)

[['6/20/2014', 'AAPL', 90.91],
 ['6/20/2014', 'MSFT', 41.68],
 ['6/20/2014', 'FB', 64.5],
 ['6/19/2014', 'AAPL', 91.86],
 ['6/19/2014', 'MSFT', 41.51],
 ['6/19/2014', 'FB', 64.34]]


Here we did not parse the first two columns in any special way, meaning we just read them in as strings. The last column we read in as `float`.

It is OK to read a date as a string, but if we want to do any operations with them (like which date was before or after, how many days are in-between, etc), we would need to convert them into some numeric values, writing our own parser like in lecture 7. Somebody already wrote a good date parser (`dateutil.parser`), so let's use that.

In [4]:
from dateutil.parser import parse
data = []
with open('files/stockprices.csv', 'r') as f: 
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        date = parse(row[0])
        symbol = row[1]
        closing_price = float(row[2]) 
        data.append([date,symbol,closing_price])
pp.pprint(data)

[[datetime.datetime(2014, 6, 20, 0, 0), 'AAPL', 90.91],
 [datetime.datetime(2014, 6, 20, 0, 0), 'MSFT', 41.68],
 [datetime.datetime(2014, 6, 20, 0, 0), 'FB', 64.5],
 [datetime.datetime(2014, 6, 19, 0, 0), 'AAPL', 91.86],
 [datetime.datetime(2014, 6, 19, 0, 0), 'MSFT', 41.51],
 [datetime.datetime(2014, 6, 19, 0, 0), 'FB', 64.34]]


Now we have nice datetime objects that come with all kind of methods, and we can do all kinds of operations with them, for example:

In [5]:
first_time = data[3][0]
later_time = data[2][0]
difference = later_time - first_time
print(difference)

1 day, 0:00:00


### Reading a "dirty" data set

It is *very* common to not start with a "clean" data set, but with something that is "dirty". For example, one value could have a different data type than other values of the same attribute, like the `n/a` here which stands for a missing value:

In [6]:
!head files/stockprices_missing.csv

6/20/2014,AAPL,90.91
6/20/2014,MSFT,41.68
6/20/2014,FB,64.5
6/19/2014,AAPL,91.86
6/19/2014,MSFT,n/a
6/19/2014,FB,64.34

If we use the code from above, it will crash because it cannot turn `n/a` into a `float`:

In [7]:
from dateutil.parser import parse
data = []
with open('files/stockprices_missing.csv', 'r') as f: 
    reader = csv.reader(f, delimiter=',')
    for row in reader:
        date = parse(row[0])
        symbol = row[1]
        closing_price = float(row[2]) 
        data.append([date,symbol,closing_price])
pp.pprint(data)

ValueError: could not convert string to float: 'n/a'

To account for malformed data like this, we could introduce `if` clauses into our code above and check explicitly for specific problems. But there is a more general way to handle this.

#### Implementing a parser

First, let's re-write our parsing problem by using functions that parse rows, telling how to parse each column:

In [8]:
def parse_row(input_row, parsers):
    """given a list of parsers (some of which may be None)
apply the appropriate one to each element of the input_row"""
    return [parser(value) if parser is not None else value for value, parser in zip(input_row, parsers)]

def parse_rows_with(reader, parsers):
    """wrap a reader to apply the parsers to each of its rows"""
    for row in reader:
        yield parse_row(row, parsers)

It works for our nice data set:

In [9]:
data = []
with open("files/stockprices.csv", "r") as f:
    reader = csv.reader(f)
    for line in parse_rows_with(reader, [parse, None, float]):
        data.append(line)
pp.pprint(data)

[[datetime.datetime(2014, 6, 20, 0, 0), 'AAPL', 90.91],
 [datetime.datetime(2014, 6, 20, 0, 0), 'MSFT', 41.68],
 [datetime.datetime(2014, 6, 20, 0, 0), 'FB', 64.5],
 [datetime.datetime(2014, 6, 19, 0, 0), 'AAPL', 91.86],
 [datetime.datetime(2014, 6, 19, 0, 0), 'MSFT', 41.51],
 [datetime.datetime(2014, 6, 19, 0, 0), 'FB', 64.34]]


But it does not work for our bad data set:

In [10]:
data = []
with open("files/stockprices_missing.csv", "r") as f:
    reader = csv.reader(f)
    for line in parse_rows_with(reader, [parse, None, float]):
        data.append(line)
pp.pprint(data)

ValueError: could not convert string to float: 'n/a'

#### Adding a helper function with a try

In [11]:
def try_or_none(f):
    """Wraps f to return None if f raises an exception. 
    Assumes f takes only one input."""
    def f_or_none(x):
        try: return f(x)
        except: return None 
    return f_or_none

# Let's re-define our parse_row function, adding the try_or_none part:
def parse_row(input_row, parsers):
    return [try_or_none(parser)(value) if parser is not None else value
for value, parser in zip(input_row, parsers)]

Now it should work, and the bad piece of data will be read in as `None`:

In [12]:
data = []
with open("files/stockprices_missing.csv", "r") as f:
    reader = csv.reader(f)
    for line in parse_rows_with(reader, [parse, None, float]):
        data.append(line)
pp.pprint(data)

[[datetime.datetime(2014, 6, 20, 0, 0), 'AAPL', 90.91],
 [datetime.datetime(2014, 6, 20, 0, 0), 'MSFT', 41.68],
 [datetime.datetime(2014, 6, 20, 0, 0), 'FB', 64.5],
 [datetime.datetime(2014, 6, 19, 0, 0), 'AAPL', 91.86],
 [datetime.datetime(2014, 6, 19, 0, 0), 'MSFT', None],
 [datetime.datetime(2014, 6, 19, 0, 0), 'FB', 64.34]]


#### Checking and deciding what to do with the bad data

The last step is now to investigate whether/where we have `None`s and decide what to do:

In [13]:
for row in data:
    if any(x is None for x in row):
        print(row)

[datetime.datetime(2014, 6, 19, 0, 0), 'MSFT', None]


Generally speaking, we have now three options:
1. get rid of this row
2. go back to the source and try to fix the bad/missing data
3. do nothing and cross our fingers

In any case, if we want our analysis to be reproducible, we *must* document what we did, and we must keep the original data set so that the cleaning and analysis can be run again the same way, leading to the same results.

Note that libraries like `pandas` have several data cleaning features built-in, like the ones developed above from scratch, which can make your life easier. Still, many data sets or problems require their unique solutions, so often you cannot rely on a standardized library or approach. 